In [1]:
from statsmodels.regression.rolling import RollingOLS
import pandas as pd
import pandas_datareader.data as web
import pandas_ta
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm 
import datetime as dt
import yfinance as yf 
import warnings
warnings.filterwarnings('ignore')

### Consulta Lista Empresas Ibov

In [2]:
# Adicionando lista de empresas listadas na Ibov
ibov = pd.read_html('https://pt.wikipedia.org/wiki/Lista_de_companhias_citadas_no_Ibovespa')[0]
# Deixando no padrão do Yfinance
ibov['Código'] = ibov['Código'] + '.SA'
codigos = ibov['Código'].unique().tolist()

### Pegando Dados Yfinance

In [3]:
fim = '2023-12-29'
# Pegando Dados dos últimos 5 Anos
inicio = pd.to_datetime(fim)-pd.DateOffset(365*5)
# Salvando Dados no dataframe
df = yf.download(tickers=codigos, start=inicio, end=fim).stack()
df.index.names = ['Data', 'Ticker']
df.columns = df.columns.str.lower()

[*********************100%%**********************]  75 of 76 completed


12 Failed downloads:
['SULA11.SA', 'BRDT3.SA', 'BTOW3.SA', 'GNDI3.SA', 'BRML3.SA', 'TIMP3.SA', 'IGTA3.SA', 'HGTX3.SA', 'VVAR3.SA', 'VIVT4.SA', 'ENBR3.SA', 'LAME4.SA']: Exception('%ticker%: No timezone found, symbol may be delisted')


### Cálculando Indicadores

* Garman-Klass Vol
* RSI
* Bollinger Bands
* ATR
* MACD
* Volume em Reais

Garman-Klass Volatility é uma medida de volatilidade que leva em consideração os preços mais altos, mais baixos, de abertura e de fechamento, e é especialmente utilizada em análises de dados intradiários no mercado financeiro.

$$
\text{Garman-Klass Volatility} = \sqrt{\frac{1}{N-1} \sum_{i=1}^{N} \left( \ln\left(\frac{H_i}{L_i}\right) \right)^2 - \frac{2}{N-1} \sum_{i=1}^{N} \left( \ln\left(\frac{C_i}{O_i}\right) \right)^2}
$$


onde:
- \(N\) é o número de períodos;
- \(H_i\) é o preço mais alto durante o período \(i\);
- \(L_i\) é o preço mais baixo durante o período \(i\);
- \(C_i\) é o preço de fechamento durante o período \(i\);
- \(O_i\) é o preço de abertura durante o período \(i\).


In [4]:
df['garman_klass_vol'] = ((np.log(df['high'])-np.log(df['low']))**2)/2-(2*np.log(2)-1)*((np.log(df['adj close'])-np.log(df['open']))**2)

### Cálculando RSI utilizando a biblioteca pandas_ta

O RSI (Relative Strength Index) é um indicador técnico usado na análise técnica de mercados financeiros, como ações, títulos e moedas. Ele foi desenvolvido por J. Welles Wilder e é utilizado para medir a força e a velocidade das mudanças de preço. O RSI é comumente utilizado para identificar condições de sobrecompra ou sobrevenda em um ativo, o que pode indicar possíveis reversões de tendência.

$$ RSI = 100 - \left( \frac{100}{1 + RS} \right) $$

RS é a média de ganhos durante o período de análise dividida pela média de perdas durante o mesmo período. O RSI gera valores entre 0 e 100. Um valor acima de 70 geralmente indica que um ativo está sobrecomprado (pode estar prestes a sofrer uma correção descendente), enquanto um valor abaixo de 30 geralmente sugere que um ativo está sobrevendido (pode estar prestes a ter uma reversão ascendente). 

In [5]:
df['rsi'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))

### Cálculando Bollinger Bands utilizando a biblioteca pandas_ta

As Bandas de Bollinger (Bollinger Bands) são um indicador técnico popular na análise de mercado financeiro, desenvolvido por John Bollinger. Esse indicador é utilizado para avaliar a volatilidade e identificar potenciais pontos de virada ou reversão de tendência em um ativo financeiro, como ações, moedas ou commodities.

São compostas por três linhas:

1 - Banda Inferior: Calculada subtraindo duas vezes o desvio padrão dos preços de fechamento da SMA.

2 - Média Móvel (linha central): Uma média móvel simples do preço de fechamento ao longo do mesmo período usado para calcular a banda superior.

3 - Banda Superior: Calculada adicionando duas vezes o desvio padrão dos preços de fechamento a uma média móvel simples (SMA) de um determinado período.

In [6]:
df['bb_inferior'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log(x), length=20).iloc[:,0])
df['bb_media'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log(x), length=20).iloc[:,1])
df['bb_superior'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log(x), length=20).iloc[:,2])

### Cálculando ATR utilizando a biblioteca pandas_ta

O ATR (Average True Range) é um indicador técnico utilizado na análise técnica dos mercados financeiros. Ele foi desenvolvido por J. Welles Wilder Jr. e tem como objetivo medir a volatilidade de um ativo ao longo de um determinado período. O ATR não fornece direção da tendência, mas sim a magnitude da volatilidade.

A fórmula básica para o cálculo do ATR envolve três etapas:

True Range (TR): É a maior diferença entre:

A diferença entre o preço mais alto e o preço mais baixo do período.
A diferença absoluta entre o preço de fechamento atual e o preço mais alto do período anterior.
A diferença absoluta entre o preço de fechamento atual e o preço mais baixo do período anterior.
A True Range é o valor máximo dessas três opções.

Média True Range (ATR): É uma média móvel exponencial (EMA) da True Range calculada ao longo de um determinado número de períodos. O período padrão é frequentemente definido como 14, mas pode ser ajustado de acordo com as preferências do analista.

O ATR é expresso como um valor absoluto em termos de pontos ou como uma porcentagem do preço atual. Quanto maior o valor do ATR, maior é a volatilidade percebida do ativo.

Os traders usam o ATR para diversas finalidades, incluindo:

Identificar níveis de stop-loss, ajustando-os de acordo com a volatilidade do mercado.
Avaliar a força de uma tendência, uma vez que picos no ATR podem indicar períodos de maior volatilidade associados a movimentos de tendência.
Comparar a volatilidade entre diferentes ativos.
O ATR é uma ferramenta versátil que pode fornecer informações valiosas sobre as condições de mercado e ajudar os traders a ajustar suas estratégias de acordo.

In [7]:
def calcula_atr(stock_data):
    atr = pandas_ta.atr(high = stock_data['high'],
                        low = stock_data['low'],
                        close = stock_data['close'],
                        length=14)
    return atr.sub(atr.mean()).div(atr.std())

df['atr'] = df.groupby(level=1, group_keys=False).apply(calcula_atr)

### Cálculando MACD utilizando a biblioteca pandas_ta

O MACD, que significa Moving Average Convergence Divergence, é um indicador técnico utilizado na análise técnica dos mercados financeiros. Ele foi desenvolvido por Gerald Appel e é amplamente utilizado para identificar a força e a direção de uma tendência, bem como para gerar sinais de compra e venda.

O MACD é calculado subtraindo a média móvel exponencial (EMA) de 26 períodos da EMA de 12 períodos. Essa operação resulta na chamada "linha MACD". Além disso, é comum calcular a EMA de 9 períodos dessa linha MACD, gerando a "linha de sinal" ou "sinal MACD".

Além das linhas mencionadas, o histograma MACD é frequentemente plotado abaixo do gráfico principal. O histograma representa a diferença entre o MACD e a linha de sinal. Se o MACD estiver acima da linha de sinal, o histograma é positivo; se estiver abaixo, o histograma é negativo.

Os sinais gerados pelo MACD incluem:

Cruzamento de Linhas: Um sinal de compra ocorre quando a linha MACD cruza acima da linha de sinal, indicando uma possível reversão de alta. Um sinal de venda ocorre quando a linha MACD cruza abaixo da linha de sinal, indicando uma possível reversão de baixa.

Divergência/Convergência: O MACD também pode ser usado para identificar divergências entre a direção do indicador e a direção dos preços. Uma divergência de alta ocorre quando o preço está fazendo novas mínimas, mas o MACD não está confirmando essas mínimas, indicando uma possível reversão de alta. Da mesma forma, uma divergência de baixa ocorre quando o preço está fazendo novas máximas, mas o MACD não está confirmando essas máximas, indicando uma possível reversão de baixa.

O MACD é uma ferramenta versátil que pode ser utilizada em diferentes prazos e em conjunto com outros indicadores para tomar decisões de negociação mais informadas.

In [8]:
def calcula_macd(close):
    macd = pandas_ta.macd(close= close, lenght = 20).iloc[:,0]
    return macd.sub(macd.mean()).div(macd.std())

df['macd'] = df.groupby(level=1, group_keys=False)['adj close'].apply(calcula_macd)

### Volume em Reais R$

Volume em Reais=Preço de Fechamento × Volume Negociado

In [9]:
df['volume_reais'] = (df['close'] * df['volume'])/1e6

## Agregando valores por MÊS

In [10]:
df

adj close      close       high        low       open  \
Data       Ticker                                                            
2019-01-02 ABEV3.SA  13.257461  16.150000  16.299999  15.400000  15.400000   
           AZUL4.SA  36.330002  36.330002  36.549999  35.450001  35.970001   
           B3SA3.SA   7.776320   9.263333   9.303333   8.893333   8.950000   
           BBAS3.SA  35.497108  48.599998  49.700001  46.090000  46.200001   
           BBDC3.SA  17.376293  22.395441  22.608315  21.005508  21.274731   
...                        ...        ...        ...        ...        ...   
2023-12-28 UGPA3.SA  26.510000  26.510000  26.590000  26.320000  26.469999   
           USIM5.SA   9.290000   9.290000   9.320000   9.200000   9.300000   
           VALE3.SA  77.199997  77.199997  77.459999  76.750000  77.000000   
           WEGE3.SA  36.910000  36.910000  37.220001  36.709999  36.840000   
           YDUQ3.SA  22.420000  22.420000  22.990000  22.320000  22.719999   

                         volume  garman_klass_vol        rsi  bb_inferior  \
Data       Ticker                                                           
2019-01-02 ABEV3.SA  18692900.0         -0.007056        NaN          NaN   
           AZUL4.SA   2795000.0          0.000429        NaN          NaN   
           B3SA3.SA  73845000.0         -0.006617        NaN          NaN   
           BBAS3.SA  14905300.0         -0.023984        NaN          NaN   
           BBDC3.SA   5374737.0         -0.013123        NaN          NaN   
...                         ...               ...        ...          ...   
2023-12-28 UGPA3.SA   3931100.0          0.000051  67.802230     3.218144   
           USIM5.SA   7530100.0          0.000084  81.124622     2.065292   
           VALE3.SA  16545800.0          0.000040  66.184579     4.264780   
           WEGE3.SA   3536300.0          0.000094  64.426760     3.527938   
           YDUQ3.SA   1983800.0          0.000369  57.165460     2.980516   

                     bb_media  bb_superior       atr      macd  volume_reais  
Data       Ticker                                                             
2019-01-02 ABEV3.SA       NaN          NaN       NaN       NaN    301.890328  
           AZUL4.SA       NaN          NaN       NaN       NaN    101.542355  
           B3SA3.SA       NaN          NaN       NaN       NaN    684.050849  
           BBAS3.SA       NaN          NaN       NaN       NaN    724.397557  
           BBDC3.SA       NaN          NaN       NaN       NaN    120.369606  
...                       ...          ...       ...       ...           ...  
2023-12-28 UGPA3.SA  3.250419     3.282693 -0.597854  1.261526    104.213462  
           USIM5.SA  2.160915     2.256539 -1.053023  1.337194     69.954629  
           VALE3.SA  4.308796     4.352812 -1.139533  0.743882   1277.335710  
           WEGE3.SA  3.574690     3.621442 -0.709877  0.738600    130.524832  
           YDUQ3.SA  3.079366     3.178216 -0.759952  0.641558     44.476796  

[79178 rows x 14 columns]

In [11]:
df.unstack('Ticker')['volume_reais'].resample('M').mean().stack('Ticker').to_frame('volume_reais')

volume_reais
Data       Ticker                
2019-01-31 ABEV3.SA    444.473942
           AZUL4.SA     71.537582
           B3SA3.SA    397.371941
           BBAS3.SA    522.876401
           BBDC3.SA     99.416926
...                           ...
2023-12-31 USIM5.SA     91.724477
           VALE3.SA   1473.209046
           WEGE3.SA    185.353146
           YDUQ3.SA     62.268580
           NTCO3.SA    165.078959

[3829 rows x 1 columns]

In [12]:
last_cols = [c for c in df.columns.unique(0) if c not in ['volume_reais', 'volume', 'open', 'close', 'high', 'low']]
last_cols

['adj close',
 'garman_klass_vol',
 'rsi',
 'bb_inferior',
 'bb_media',
 'bb_superior',
 'atr',
 'macd']

In [13]:
data = (pd.concat([df.unstack('Ticker')['volume_reais'].resample('M').mean().stack('Ticker').to_frame('volume_reais'),
df.unstack()[last_cols].resample('M').last().stack('Ticker')], axis=1)).dropna()

data

volume_reais  adj close  garman_klass_vol        rsi  \
Data       Ticker                                                           
2019-02-28 ABEV3.SA    429.137554  14.144027         -0.022377  43.687162   
           AZUL4.SA     89.182815  37.650002          0.000287  51.891211   
           B3SA3.SA    376.232682   9.175446         -0.014121  58.961003   
           BBAS3.SA    646.010074  37.373707         -0.041640  48.879471   
           BBDC3.SA    107.426286  18.734192         -0.030624  48.024904   
...                           ...        ...               ...        ...   
2023-12-31 USIM5.SA     91.724477   9.290000          0.000084  81.124622   
           VALE3.SA   1473.209046  77.199997          0.000040  66.184579   
           WEGE3.SA    185.353146  36.910000          0.000094  64.426760   
           YDUQ3.SA     62.268580  22.420000          0.000369  57.165460   
           NTCO3.SA    165.078959  16.889999          0.000401  58.098499   

                     bb_inferior  bb_media  bb_superior       atr      macd  
Data       Ticker                                                            
2019-02-28 ABEV3.SA     2.676277  2.711787     2.747297  0.514885  0.515452  
           AZUL4.SA     3.599754  3.635583     3.671411 -0.526987  0.316224  
           B3SA3.SA     2.147163  2.200531     2.253898 -1.452605  0.624334  
           BBAS3.SA     3.605378  3.653805     3.702233  0.983837  0.572881  
           BBDC3.SA     2.934031  2.967109     3.000188  1.005420  0.622266  
...                          ...       ...          ...       ...       ...  
2023-12-31 USIM5.SA     2.065292  2.160915     2.256539 -1.053023  1.337194  
           VALE3.SA     4.264780  4.308796     4.352812 -1.139533  0.743882  
           WEGE3.SA     3.527938  3.574690     3.621442 -0.709877  0.738600  
           YDUQ3.SA     2.980516  3.079366     3.178216 -0.759952  0.641558  
           NTCO3.SA     2.774688  2.817838     2.860987 -1.211478  0.381969  

[3765 rows x 9 columns]